<a href="https://colab.research.google.com/github/mantoan-thi/-Python-for-investments-/blob/main/Stock_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas-datareader

In [82]:
import pandas as pd
from pandas_datareader import data
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

In [112]:
tickers = ['RRRP3.SA','ALPA4.SA','ABEV3.SA','AMER3.SA','ASAI3.SA','AZUL4.SA']
          #'B3SA3.SA','BIDI11.SA','BPAN4.SA','BBSE3.SA','BRML3.SA','BBDC3.SA',
          #'BBDC4.SA','BRAP4.SA','BBAS3.SA','BRKM5.SA','BRFS3.SA']

In [123]:
def coletar(tickers):
    start_date = '2009-01-01'
    end_date = '2016-12-31'
    dados = data.DataReader(tickers, 'yahoo', start_date)['Adj Close']
    dataset = pd.DataFrame(dados)
    dataset.reset_index(inplace=True)
    dataset.rename(columns={'Date': 'ds','Adj Close':'y'},inplace =True)
    return dataset

def previsao(dados):
    prior_scale=0.2
    m = Prophet(changepoint_prior_scale=prior_scale)
    m.fit(dados)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    #forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    return forecast['yhat'].iloc[-31:]

In [124]:
def gerar_analise():
  lista = []
  print('Carteira Teórica do IBovespa\n')
  print('Quantidade de ativos: ',len(tickers),'\n')
  for ativo in tickers:
      print('Analisando: ',ativo)
      dataset = previsao(coletar(ativo))
      var = dataset[-1:].values[0]/dataset[:1].values[0]-1
      lista.append([ativo,var])
  new_lista = pd.DataFrame(lista,columns=['Ativo','Percentual'])
  Ativo_compra = new_lista[new_lista['Percentual'] > 0]
  Ativo_venda = new_lista[new_lista['Percentual'] < 0]
  Ativo_compra.sort_values(by=['Percentual'],ascending=False,inplace=True)
  Ativo_venda.sort_values(by=['Percentual'],inplace=True)
  Ativo_compra['Percentual'] = Ativo_compra['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  Ativo_venda['Percentual'] = Ativo_venda['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  return Ativo_compra,Ativo_venda

In [ ]:
Ativo_compra, Ativo_venda = gerar_analise()

Carteira Teórica do IBovespa

Quantidade de ativos:  6 

Analisando:  RRRP3.SA


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  ALPA4.SA


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  ABEV3.SA


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  AMER3.SA


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
